# SQLAlchemy
https://www.wakuwakubank.com/posts/277-python-sqlalchemy/


In [1]:
# pip install python-dotenv

In [2]:
# pip install mysqlclient

In [3]:
import os
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, scoped_session
from sqlalchemy import desc
from dotenv import load_dotenv
load_dotenv()

True

## DB接続に必要な設定

Sessionの生成方法には以下のような種類がある。  
https://qiita.com/tosizo/items/86d3c60a4bb70eb1656e
- Sessionクラスによる生成（基本）
- sessionmakerによる生成
- scoped_sessionによる生成
- scoped_sessionによる生成（ORM編）

In [4]:
# mysqlの設定の場合
DATABASE = 'mysql+mysqldb'
USER = os.getenv('DB_USERNAME')
PASSWORD = os.getenv('DB_PASSWORD')
HOST = os.getenv('DB_HOST')
PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_DATABASE')
CHARSET_TYPE = "utf8"
DB_CONNECT_URL = f"{DATABASE}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}?charset={CHARSET_TYPE}"

In [5]:
# ローカル用のDB設定
DB_CONNECT_URL = 'sqlite:///db.sqlite3'

In [6]:
# engineの設定 接続用のインスタンスを作成（create_engine関数がEngineインスタンスを返す）
engine = create_engine(DB_CONNECT_URL,echo=True)

# DBに対してORM操作するときに利用
# Sessionを通じて操作を行う
session = scoped_session(
    sessionmaker(autocommit=False, autoflush=False, bind=engine)
)

# 各modelで利用
# classとDBをMapping
Base = declarative_base()

## テーブルの作成
metadataを使うと、既存のデータベースからテーブル定義をもってきたりできる

### （１）Userのマスターテーブル

In [7]:
from sqlalchemy import Column, Integer, String, DateTime, Sequence
from datetime import datetime
import sys

class User(Base):
    """
    UserModel
    """
    __tablename__ = 'users'
    __table_args__ = {
        'comment': 'ユーザー情報のマスターテーブル'
    }
    id = Column('id', Integer, primary_key=True, autoincrement=True)
    name = Column('name', String(200))
    age = Column('age', Integer)
    email = Column('email', String(100))
    created_at = Column('created', DateTime, default=datetime.now, nullable=False)
    updated_at = Column('modified', DateTime, default=datetime.now, nullable=False)
    
Base.metadata.create_all(bind=engine)

2022-10-26 22:00:39,602 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-26 22:00:39,603 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-10-26 22:00:39,603 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-26 22:00:39,607 INFO sqlalchemy.engine.Engine COMMIT


### （２）Logのマスターテーブル

In [8]:
class AppLog(Base):
    """
    AppLogModel
    """
    __tablename__ = 'logs'
    __table_args__ = {
        'comment': 'アプリのログテーブル'
    }
    log_id = Column('id', Integer, primary_key=True, autoincrement=True)
    user_id = Column('user_id', Integer)
    event_id = Column('event_id', Integer)
    created_at = Column('created', DateTime, default=datetime.now, nullable=False)
    
Base.metadata.create_all(bind=engine)

2022-10-26 22:00:39,654 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-26 22:00:39,655 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-10-26 22:00:39,656 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-26 22:00:39,660 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("logs")
2022-10-26 22:00:39,660 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-26 22:00:39,662 INFO sqlalchemy.engine.Engine COMMIT


## ORM操作
https://zenn.dev/shimakaze_soft/articles/6e5e47851459f5
https://qiita.com/simonritchie/items/705148154f810ee308e2


データベースとマッピングされたオブジェクトを利用して、データベースを操作する技法

In [9]:
# usersテーブルのレコードを全て取得する
# SELECT * FROM users; と同じ
users = session.query(User).all()
for user in users:
    print(user.id,user.name,user.age,user.email,user.created_at,user.updated_at)

2022-10-26 22:00:39,700 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-26 22:00:39,701 INFO sqlalchemy.engine.Engine SELECT users.created AS users_created, users.modified AS users_modified, users.id AS users_id, users.name AS users_name, users.age AS users_age, users.email AS users_email 
FROM users
2022-10-26 22:00:39,702 INFO sqlalchemy.engine.Engine [generated in 0.00057s] ()
4 user0 None None 2022-10-26 21:47:46.185966 2022-10-26 21:47:46.185968
5 user1 None None 2022-10-26 21:47:46.186926 2022-10-26 21:47:46.186927
6 user2 None None 2022-10-26 21:47:46.187619 2022-10-26 21:47:46.187621
7 user3 None None 2022-10-26 21:47:46.188409 2022-10-26 21:47:46.188410
8 user4 None None 2022-10-26 21:47:46.188952 2022-10-26 21:47:46.188954
9 user5 None None 2022-10-26 21:47:46.191125 2022-10-26 21:47:46.191127
10 user6 None None 2022-10-26 21:47:46.192753 2022-10-26 21:47:46.192755
11 user7 None None 2022-10-26 21:47:46.193680 2022-10-26 21:47:46.193682
12 user8 None None 2022-10-26 21

In [10]:
# 初期化
# 全データ削除
session.query(User).delete()
session.commit()

2022-10-26 22:00:39,741 INFO sqlalchemy.engine.Engine DELETE FROM users
2022-10-26 22:00:39,743 INFO sqlalchemy.engine.Engine [generated in 0.00119s] ()
2022-10-26 22:00:39,752 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
# logsテーブルのレコードを全て取得する
logs = session.query(AppLog).all()
print("")
print("log_id","user_id","event_id","created_at")
for log in logs:
    print(log.log_id,log.user_id,log.event_id,log.created_at)

2022-10-26 22:00:39,857 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-26 22:00:39,859 INFO sqlalchemy.engine.Engine SELECT logs.id AS logs_id, logs.created AS logs_created, logs.user_id AS logs_user_id, logs.event_id AS logs_event_id 
FROM logs
2022-10-26 22:00:39,859 INFO sqlalchemy.engine.Engine [generated in 0.00074s] ()

log_id user_id event_id created_at
1 user8 None 2022-10-26 21:48:02.038021
2 user5 None 2022-10-26 21:48:02.057491
3 user5 None 2022-10-26 21:48:02.061237
4 user0 None 2022-10-26 21:48:02.063866
5 user0 None 2022-10-26 21:48:02.067640
6 user8 None 2022-10-26 21:48:02.069681
7 user2 None 2022-10-26 21:48:02.071780
8 user3 None 2022-10-26 21:48:02.072882
9 user1 None 2022-10-26 21:48:02.076569
10 user9 None 2022-10-26 21:48:02.077896
11 user8 None 2022-10-26 21:48:02.079076
12 user0 None 2022-10-26 21:48:02.081889
13 user3 None 2022-10-26 21:48:02.084275
14 user1 None 2022-10-26 21:48:02.085044
15 user5 None 2022-10-26 21:48:02.085858
16 user10 None 2022-10-

In [12]:
# 初期化
# 全データ削除
session.query(AppLog).delete()
session.commit()

2022-10-26 22:00:39,901 INFO sqlalchemy.engine.Engine DELETE FROM logs
2022-10-26 22:00:39,902 INFO sqlalchemy.engine.Engine [generated in 0.00056s] ()
2022-10-26 22:00:39,912 INFO sqlalchemy.engine.Engine COMMIT


## CRUD処理

### Create (Insert)

In [13]:
# １つ挿入する場合
user = User()
user.name = 'sample-name'
# insert
session.add(user)

# 複数挿入する場合
add_list = [
    User(name='orange'),
    User(name='melon'),
]
# insert
session.add_all(instances=add_list)

# 以下で使いそうなものをforで生成しておく
add_list = []
for i in range(0,11):
    user = User()
    user.name = 'user' + str(i)
    add_list.append(user)
# insert
session.add_all(instances=add_list)

# commitでデータベースへ反映
session.commit()

2022-10-26 22:00:39,960 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-26 22:00:39,962 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age, email, created, modified) VALUES (?, ?, ?, ?, ?)
2022-10-26 22:00:39,963 INFO sqlalchemy.engine.Engine [generated in 0.00111s] ('sample-name', None, None, '2022-10-26 22:00:39.961979', '2022-10-26 22:00:39.961985')
2022-10-26 22:00:39,980 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age, email, created, modified) VALUES (?, ?, ?, ?, ?)
2022-10-26 22:00:39,981 INFO sqlalchemy.engine.Engine [cached since 0.01983s ago] ('orange', None, None, '2022-10-26 22:00:39.980566', '2022-10-26 22:00:39.980573')
2022-10-26 22:00:39,982 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age, email, created, modified) VALUES (?, ?, ?, ?, ?)
2022-10-26 22:00:39,983 INFO sqlalchemy.engine.Engine [cached since 0.02124s ago] ('melon', None, None, '2022-10-26 22:00:39.982331', '2022-10-26 22:00:39.982333')
2022-10-26 22:00:39,983 INFO sqlalc

In [14]:
# usersテーブルのレコードを全て取得する
users = session.query(User).all()
print("")
print("id","name","age","email","created_at","updated_at")
for user in users:
    print(user.id,user.name,user.age,user.email,user.created_at,user.updated_at)

2022-10-26 22:00:40,113 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-26 22:00:40,114 INFO sqlalchemy.engine.Engine SELECT users.created AS users_created, users.modified AS users_modified, users.id AS users_id, users.name AS users_name, users.age AS users_age, users.email AS users_email 
FROM users
2022-10-26 22:00:40,114 INFO sqlalchemy.engine.Engine [cached since 0.413s ago] ()

id name age email created_at updated_at
1 sample-name None None 2022-10-26 22:00:39.961979 2022-10-26 22:00:39.961985
2 orange None None 2022-10-26 22:00:39.980566 2022-10-26 22:00:39.980573
3 melon None None 2022-10-26 22:00:39.982331 2022-10-26 22:00:39.982333
4 user0 None None 2022-10-26 22:00:39.983708 2022-10-26 22:00:39.983711
5 user1 None None 2022-10-26 22:00:39.984717 2022-10-26 22:00:39.984718
6 user2 None None 2022-10-26 22:00:39.985742 2022-10-26 22:00:39.985744
7 user3 None None 2022-10-26 22:00:39.986681 2022-10-26 22:00:39.986683
8 user4 None None 2022-10-26 22:00:39.987728 2022-10-26 

In [15]:
import random
add_list = []
for i in range(0,20):
    log = AppLog()
    log.user_id = 'user' + str(random.randint(0, 10))
    add_list.append(log)
# insert
session.add_all(instances=add_list)
session.commit()

2022-10-26 22:00:40,211 INFO sqlalchemy.engine.Engine INSERT INTO logs (user_id, event_id, created) VALUES (?, ?, ?)
2022-10-26 22:00:40,213 INFO sqlalchemy.engine.Engine [generated in 0.00219s] ('user7', None, '2022-10-26 22:00:40.211336')
2022-10-26 22:00:40,223 INFO sqlalchemy.engine.Engine INSERT INTO logs (user_id, event_id, created) VALUES (?, ?, ?)
2022-10-26 22:00:40,223 INFO sqlalchemy.engine.Engine [cached since 0.01231s ago] ('user4', None, '2022-10-26 22:00:40.222985')
2022-10-26 22:00:40,224 INFO sqlalchemy.engine.Engine INSERT INTO logs (user_id, event_id, created) VALUES (?, ?, ?)
2022-10-26 22:00:40,225 INFO sqlalchemy.engine.Engine [cached since 0.01372s ago] ('user8', None, '2022-10-26 22:00:40.224695')
2022-10-26 22:00:40,225 INFO sqlalchemy.engine.Engine INSERT INTO logs (user_id, event_id, created) VALUES (?, ?, ?)
2022-10-26 22:00:40,226 INFO sqlalchemy.engine.Engine [cached since 0.01506s ago] ('user8', None, '2022-10-26 22:00:40.225348')
2022-10-26 22:00:40,227 

In [16]:
# logsテーブルのレコードを全て取得する
logs = session.query(AppLog).all()
print("")
print("log_id","user_id","event_id","created_at")
for log in logs:
    print(log.log_id,log.user_id,log.event_id,log.created_at)

2022-10-26 22:00:40,293 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-26 22:00:40,294 INFO sqlalchemy.engine.Engine SELECT logs.id AS logs_id, logs.created AS logs_created, logs.user_id AS logs_user_id, logs.event_id AS logs_event_id 
FROM logs
2022-10-26 22:00:40,295 INFO sqlalchemy.engine.Engine [cached since 0.436s ago] ()

log_id user_id event_id created_at
1 user7 None 2022-10-26 22:00:40.211336
2 user4 None 2022-10-26 22:00:40.222985
3 user8 None 2022-10-26 22:00:40.224695
4 user8 None 2022-10-26 22:00:40.225348
5 user6 None 2022-10-26 22:00:40.227489
6 user1 None 2022-10-26 22:00:40.228272
7 user6 None 2022-10-26 22:00:40.229215
8 user3 None 2022-10-26 22:00:40.231044
9 user0 None 2022-10-26 22:00:40.232192
10 user10 None 2022-10-26 22:00:40.235695
11 user9 None 2022-10-26 22:00:40.237064
12 user3 None 2022-10-26 22:00:40.238320
13 user5 None 2022-10-26 22:00:40.239464
14 user8 None 2022-10-26 22:00:40.240326
15 user0 None 2022-10-26 22:00:40.241113
16 user10 None 2022-

In [17]:
# 最後にsessionを閉じ処理
session.close()

2022-10-26 22:00:40,351 INFO sqlalchemy.engine.Engine ROLLBACK


### Read (Select)

In [18]:
# usersテーブルのレコードを全て取得する
users = session.query(User).all()
print("")
print("id","name","age","email","created_at","updated_at")
for user in users:
    print(user.id,user.name,user.age,user.email,user.created_at,user.updated_at)

2022-10-26 22:00:40,412 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-26 22:00:40,413 INFO sqlalchemy.engine.Engine SELECT users.created AS users_created, users.modified AS users_modified, users.id AS users_id, users.name AS users_name, users.age AS users_age, users.email AS users_email 
FROM users
2022-10-26 22:00:40,414 INFO sqlalchemy.engine.Engine [cached since 0.7125s ago] ()

id name age email created_at updated_at
1 sample-name None None 2022-10-26 22:00:39.961979 2022-10-26 22:00:39.961985
2 orange None None 2022-10-26 22:00:39.980566 2022-10-26 22:00:39.980573
3 melon None None 2022-10-26 22:00:39.982331 2022-10-26 22:00:39.982333
4 user0 None None 2022-10-26 22:00:39.983708 2022-10-26 22:00:39.983711
5 user1 None None 2022-10-26 22:00:39.984717 2022-10-26 22:00:39.984718
6 user2 None None 2022-10-26 22:00:39.985742 2022-10-26 22:00:39.985744
7 user3 None None 2022-10-26 22:00:39.986681 2022-10-26 22:00:39.986683
8 user4 None None 2022-10-26 22:00:39.987728 2022-10-26

### Update (Update)

In [19]:
# 番最初のレコードを取得し、そのレコードのnamaカラムを変更して更新する
user = session.query(User).first()
user.name = 'user_update_test'
session.commit()

2022-10-26 22:00:40,505 INFO sqlalchemy.engine.Engine SELECT users.created AS users_created, users.modified AS users_modified, users.id AS users_id, users.name AS users_name, users.age AS users_age, users.email AS users_email 
FROM users
 LIMIT ? OFFSET ?
2022-10-26 22:00:40,506 INFO sqlalchemy.engine.Engine [generated in 0.00107s] (1, 0)
2022-10-26 22:00:40,511 INFO sqlalchemy.engine.Engine UPDATE users SET name=? WHERE users.id = ?
2022-10-26 22:00:40,511 INFO sqlalchemy.engine.Engine [generated in 0.00070s] ('user_update_test', 1)
2022-10-26 22:00:40,520 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
# usersテーブルのレコードを全て取得する
users = session.query(User).all()
print("")
print("id","name","age","email","created_at","updated_at")
for user in users:
    print(user.id,user.name,user.age,user.email,user.created_at,user.updated_at)

2022-10-26 22:00:40,609 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-26 22:00:40,610 INFO sqlalchemy.engine.Engine SELECT users.created AS users_created, users.modified AS users_modified, users.id AS users_id, users.name AS users_name, users.age AS users_age, users.email AS users_email 
FROM users
2022-10-26 22:00:40,611 INFO sqlalchemy.engine.Engine [cached since 0.9099s ago] ()

id name age email created_at updated_at
1 user_update_test None None 2022-10-26 22:00:39.961979 2022-10-26 22:00:39.961985
2 orange None None 2022-10-26 22:00:39.980566 2022-10-26 22:00:39.980573
3 melon None None 2022-10-26 22:00:39.982331 2022-10-26 22:00:39.982333
4 user0 None None 2022-10-26 22:00:39.983708 2022-10-26 22:00:39.983711
5 user1 None None 2022-10-26 22:00:39.984717 2022-10-26 22:00:39.984718
6 user2 None None 2022-10-26 22:00:39.985742 2022-10-26 22:00:39.985744
7 user3 None None 2022-10-26 22:00:39.986681 2022-10-26 22:00:39.986683
8 user4 None None 2022-10-26 22:00:39.987728 2022-

In [21]:
session.close()

2022-10-26 22:00:40,652 INFO sqlalchemy.engine.Engine ROLLBACK


### Delete (Delete)

In [22]:
user = session.query(User).first()

session.delete(user)
session.commit()

2022-10-26 22:00:40,725 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-26 22:00:40,726 INFO sqlalchemy.engine.Engine SELECT users.created AS users_created, users.modified AS users_modified, users.id AS users_id, users.name AS users_name, users.age AS users_age, users.email AS users_email 
FROM users
 LIMIT ? OFFSET ?
2022-10-26 22:00:40,727 INFO sqlalchemy.engine.Engine [cached since 0.2228s ago] (1, 0)
2022-10-26 22:00:40,737 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2022-10-26 22:00:40,738 INFO sqlalchemy.engine.Engine [generated in 0.00106s] (1,)
2022-10-26 22:00:40,749 INFO sqlalchemy.engine.Engine COMMIT


In [23]:
# usersテーブルのレコードを全て取得する
users = session.query(User).all()
print("")
print("id","name","age","email","created_at","updated_at")
for user in users:
    print(user.id,user.name,user.age,user.email,user.created_at,user.updated_at)

2022-10-26 22:00:40,789 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-26 22:00:40,790 INFO sqlalchemy.engine.Engine SELECT users.created AS users_created, users.modified AS users_modified, users.id AS users_id, users.name AS users_name, users.age AS users_age, users.email AS users_email 
FROM users
2022-10-26 22:00:40,791 INFO sqlalchemy.engine.Engine [cached since 1.089s ago] ()

id name age email created_at updated_at
2 orange None None 2022-10-26 22:00:39.980566 2022-10-26 22:00:39.980573
3 melon None None 2022-10-26 22:00:39.982331 2022-10-26 22:00:39.982333
4 user0 None None 2022-10-26 22:00:39.983708 2022-10-26 22:00:39.983711
5 user1 None None 2022-10-26 22:00:39.984717 2022-10-26 22:00:39.984718
6 user2 None None 2022-10-26 22:00:39.985742 2022-10-26 22:00:39.985744
7 user3 None None 2022-10-26 22:00:39.986681 2022-10-26 22:00:39.986683
8 user4 None None 2022-10-26 22:00:39.987728 2022-10-26 22:00:39.987729
9 user5 None None 2022-10-26 22:00:39.988864 2022-10-26 22:00:

In [24]:
session.close()

2022-10-26 22:00:40,833 INFO sqlalchemy.engine.Engine ROLLBACK


## WHERE

In [25]:
# SELECT * FROM logs WHERE user_id = 'user3';
logs = session.query(AppLog).filter(AppLog.user_id == "user3").all()
print("")
print("id","name","age","email","created_at","updated_at")
for log in logs:
    print(log.log_id,log.user_id,log.event_id,log.created_at)

2022-10-26 22:00:40,907 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-26 22:00:40,909 INFO sqlalchemy.engine.Engine SELECT logs.id AS logs_id, logs.created AS logs_created, logs.user_id AS logs_user_id, logs.event_id AS logs_event_id 
FROM logs 
WHERE logs.user_id = ?
2022-10-26 22:00:40,909 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ('user3',)

id name age email created_at updated_at
8 user3 None 2022-10-26 22:00:40.231044
12 user3 None 2022-10-26 22:00:40.238320
18 user3 None 2022-10-26 22:00:40.244142


In [26]:
# SELECT * FROM logs WHERE user_id = 'user3';
logs = session.query(AppLog).filter(AppLog.user_id == "user8").all()
print("")
print("id","name","age","email","created_at","updated_at")
for log in logs:
    print(log.log_id,log.user_id,log.event_id,log.created_at)

2022-10-26 22:00:40,969 INFO sqlalchemy.engine.Engine SELECT logs.id AS logs_id, logs.created AS logs_created, logs.user_id AS logs_user_id, logs.event_id AS logs_event_id 
FROM logs 
WHERE logs.user_id = ?
2022-10-26 22:00:40,970 INFO sqlalchemy.engine.Engine [cached since 0.06109s ago] ('user8',)

id name age email created_at updated_at
3 user8 None 2022-10-26 22:00:40.224695
4 user8 None 2022-10-26 22:00:40.225348
14 user8 None 2022-10-26 22:00:40.240326


## LIMIT

In [27]:
# SELECT * FROM logs LIMIT 6;
logs = session.query(AppLog).limit(6).all()

print("")
print("id","name","age","email","created_at","updated_at")
for log in logs:
    print(log.log_id,log.user_id,log.event_id,log.created_at)

2022-10-26 22:00:41,017 INFO sqlalchemy.engine.Engine SELECT logs.id AS logs_id, logs.created AS logs_created, logs.user_id AS logs_user_id, logs.event_id AS logs_event_id 
FROM logs
 LIMIT ? OFFSET ?
2022-10-26 22:00:41,018 INFO sqlalchemy.engine.Engine [generated in 0.00081s] (6, 0)

id name age email created_at updated_at
1 user7 None 2022-10-26 22:00:40.211336
2 user4 None 2022-10-26 22:00:40.222985
3 user8 None 2022-10-26 22:00:40.224695
4 user8 None 2022-10-26 22:00:40.225348
5 user6 None 2022-10-26 22:00:40.227489
6 user1 None 2022-10-26 22:00:40.228272


## IN
INは遅くなるので、使いすぎない方が良い

In [28]:
# SELECT * FROM users WHERE name IN("taro", "jiro", "ichiro");
names = ['user1', 'user2', 'user3']
logs = session.query(AppLog).filter(AppLog.user_id.in_(names)).all()
print("")
print("id","name","age","email","created_at","updated_at")
for log in logs:
    print(log.log_id,log.user_id,log.event_id,log.created_at)

2022-10-26 22:00:41,060 INFO sqlalchemy.engine.Engine SELECT logs.id AS logs_id, logs.created AS logs_created, logs.user_id AS logs_user_id, logs.event_id AS logs_event_id 
FROM logs 
WHERE logs.user_id IN (?, ?, ?)
2022-10-26 22:00:41,061 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ('user1', 'user2', 'user3')

id name age email created_at updated_at
6 user1 None 2022-10-26 22:00:40.228272
8 user3 None 2022-10-26 22:00:40.231044
12 user3 None 2022-10-26 22:00:40.238320
18 user3 None 2022-10-26 22:00:40.244142
20 user2 None 2022-10-26 22:00:40.245834


## ORDER BY

In [29]:
# SELECT * FROM users ORDER BY users.created_at DESC;
users = session.query(User).order_by(desc(User.created_at)).all()
print("")
print("id","name","age","email","created_at","updated_at")
for user in users:
    print(user.id,user.name,user.age,user.email,user.created_at,user.updated_at)

2022-10-26 22:00:41,109 INFO sqlalchemy.engine.Engine SELECT users.created AS users_created, users.modified AS users_modified, users.id AS users_id, users.name AS users_name, users.age AS users_age, users.email AS users_email 
FROM users ORDER BY users.created DESC
2022-10-26 22:00:41,112 INFO sqlalchemy.engine.Engine [generated in 0.00330s] ()

id name age email created_at updated_at
14 user10 None None 2022-10-26 22:00:39.993904 2022-10-26 22:00:39.993905
13 user9 None None 2022-10-26 22:00:39.992659 2022-10-26 22:00:39.992660
12 user8 None None 2022-10-26 22:00:39.991964 2022-10-26 22:00:39.991966
11 user7 None None 2022-10-26 22:00:39.991259 2022-10-26 22:00:39.991260
10 user6 None None 2022-10-26 22:00:39.990496 2022-10-26 22:00:39.990498
9 user5 None None 2022-10-26 22:00:39.988864 2022-10-26 22:00:39.988865
8 user4 None None 2022-10-26 22:00:39.987728 2022-10-26 22:00:39.987729
7 user3 None None 2022-10-26 22:00:39.986681 2022-10-26 22:00:39.986683
6 user2 None None 2022-10-26 2

## DISTINCT


In [30]:
from sqlalchemy import distinct

In [31]:
# SELECT DISTINCT name from users;
user_name = session.query(distinct(AppLog.user_id)).all()
print("")
print("log_user_name")
for user in user_name:
    print(user[0])

2022-10-26 22:00:41,206 INFO sqlalchemy.engine.Engine SELECT DISTINCT logs.user_id 
FROM logs
2022-10-26 22:00:41,207 INFO sqlalchemy.engine.Engine [generated in 0.00178s] ()

log_user_name
user7
user4
user8
user6
user1
user3
user0
user10
user9
user5
user2


In [32]:
session.close()

2022-10-26 22:00:41,258 INFO sqlalchemy.engine.Engine ROLLBACK


## JOIN (INNER JOIN)

In [ ]:
# SELECT * from users inner join user_socials on users.id = user_socials.id;
user_logs = session.query(User, AppLog)\
    .join(AppLog, User.id == AppLog.user_id)\
    .all()
for user_log in user_logs:
    print(user_log.users_name)

## LEFT JOIN

In [ ]:
# SELECT * from users letf outer join user_socials on users.id = user_socials.id;
user_logs = session.query(AppLog,User).outerjoin(User, AppLog.user_id==User.id).all()
print("")
# print("id","user_id","event_id","created_at")
for user_log in user_logs:
    # print(user_log)
    print(
        # user_log.AppLog.created_at,
        # user_log.AppLog.user_id,
        user_log.User.id
        # user_log.User.name
    )

In [ ]:
user_logs.column_descriptions

## UNION

In [ ]:
SELECT * FROM tags
UNION
SELECT * FROM genres;

In [ ]:
tag_genre = session.query(Tag).\
    union(session.query(Genre)).\
    all()

## UNION ALL

In [ ]:
SELECT * FROM tags
UNION ALL
SELECT * FROM genres;

In [ ]:
tag_genre = session.query(Tag).\
    union_all(session.query(Genre)).\
    all()

## 直接SQL文を実行
ORMが生成したSQLが処理の負担になり、パフォーマンスの低下を招くこともあるため、直接SQLを入力して実行することもできる

In [ ]:
# 直接SQLを実行
result = Engine.execute("select * from users")

for row in result:
    print(row)

## 吐き出されるSQLを取得
ORMを実行した際の実際に生成されたSQLのログを取得することもできる

In [ ]:
user_name = session.query(User)
sql_statement = user_name.statement

print(sql_statement)